## Entropia e Information Gain negli alberi decisionali

L'entropia, nella teoria dell'informazione, è una misura dell'**incertezza** associato ad una variabile casuale. In questo contesto, la parola "entropia" si riferisce solitamente all'[Entropia di Shannon](https://en.wikipedia.org/wiki/Entropy_(information_theory)), che quantifica, nel senso di un valore atteso, l'informazione contenuta in un messaggio, solitamente espressa in bit. Equivalentemente, l'entropia di Shannon è una misura del contenuto informativo medio che manca quando non si conosce il valore della variabile casuale.

<img src="https://miro.medium.com/max/565/1*M15RZMSk8nGEyOnD8haF-A.png" />

La misura di entropia si applica ad un dataset ma è relativa ad una singola feature.
Vediamo i due estremi in cui l'entropia è pari a zero in quanto non c'è incertezza. 
Se entrambi i valori, - e +, sono presenti in egual numero, allora l'entropia è massima, =1, in quanto indica la presenza di massima incertezza.

In generale è possibile calcolare l'entropia di una variabile casuale discreta come: 

<img src="https://wikimedia.org/api/rest_v1/media/math/render/svg/b29bc7ca1bec9b7b9aa03ebf2332c61e8fd1f897" />

Dove $ P_{i} $ è la probabilità di appartenenza ad una delle classi della variabile casuale $ X $ e il logaritmo è in base 2 (base b, b=numero di classi ottenute). Strettamente connesso a questo concetto è il concetto di **Information Gain** (misura utilizzata per fare la scelta di split data dall'entropia), che viene poi effettivamente utilizzato come criterio per decidere le feature su cui costruire gli split di un albero decisionale. È possibile calcolare l'information gain in questa maniera: 

<img src="https://wikimedia.org/api/rest_v1/media/math/render/svg/ce1cc627a4c7795af67661345fb6d544ac53a31e" />

in cui $ H(T) $ misura l'entropia dello split _precedente_ mentre $ H(T|a) $ misura l'entropia allo split attuale _condizionato_ da un'altra variabile. Si calcola l'information gain per ogni feature a nostra disposizione e si sceglie quella che ci dà information gain più alto. Il concetto alla base dell'Information Gain è che una caratteristica con un alto guadagno informativo contribuirà maggiormente a separare le classi nei dati rispetto ad una caratteristica con un basso guadagno informativo. La feature selezionata contribuirà a ridurre l'entropia di $ T $ di un piccolo pezzo. 

In [9]:
import pandas as pd
import numpy as np
import math

dataset = [
    ['Giuseppe', 37, 0, 1, 1, 1],
    ['Carla', 37, 1, 1, 0, 1],
    ['Giulia', 23, 1, 1, 0, 1],
    ['Francesca', 26, 0, 0, 1, 0],
    ['Gianni', 30, 1, 1, 1, 1],
    ['Vincenzo', 25, 1, 0, 0, 1],
    ['Daniele', 21, 1, 1, 0, 1],
    ['Davide', 42, 1, 1, 0, 1],
    ['Antonio', 21, 1, 1, 0, 0],
    ['Marianna', 47, 1, 0, 1, 0],
    ['Gaetano', 72, 1, 0, 0, 1],
    ['Francesco', 24, 1, 1, 1, 0],
    ['Giovanni', 19, 0, 1, 0, 1],
    ['Fabio', 38, 1, 0, 0, 1],
    ['Massimo', 31, 0, 0, 1, 0]
]

columns = ["name", "age", "parmigiana", "polenta", "sushi", "meridionale"]

In [10]:
df=pd.DataFrame(dataset,columns=columns)
df.head()

,name,age,parmigiana,polenta,sushi,meridionale
0,Giuseppe,37,0,1,1,1
1,Carla,37,1,1,0,1
2,Giulia,23,1,1,0,1
3,Francesca,26,0,0,1,0
4,Gianni,30,1,1,1,1


In [11]:
#uniformiamo le features, creiamo una nuova feature nel dataset per uniformare age in modo che sia binaria come le altre

values=[1 if n>df.age.median() else 0 for n in df.age]
values

[1, 1, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 0, 1, 1]

In [12]:
#inserisce una colonna nel dataframe in una locazione specifica
df.insert(2, "age_gt_median", values)
print(df.info())
df.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15 entries, 0 to 14
Data columns (total 7 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   name           15 non-null     object
 1   age            15 non-null     int64 
 2   age_gt_median  15 non-null     int64 
 3   parmigiana     15 non-null     int64 
 4   polenta        15 non-null     int64 
 5   sushi          15 non-null     int64 
 6   meridionale    15 non-null     int64 
dtypes: int64(6), object(1)
memory usage: 968.0+ bytes
None


,name,age,age_gt_median,parmigiana,polenta,sushi,meridionale
0,Giuseppe,37,1,0,1,1,1
1,Carla,37,1,1,1,0,1
2,Giulia,23,0,1,1,0,1
3,Francesca,26,0,0,0,1,0
4,Gianni,30,0,1,1,1,1


In [13]:
def entropy(column):
    counts=np.bincount(column) #conto quanti 1 e quanti 0
    probabilities= counts/len(column)

    entropy=0

    for prob in  probabilities:
        if prob >0:
            entropy+= prob*math.log(prob,2)
            #guardiamo la formula per il calcolo, aggiugiamo a entropy prob*log(in base due, della probabilità)

    return -entropy

In [15]:
entropy(values)
#molot alto
np.bincount(values)

array([8, 7])

In [17]:
for c in df.columns[2:]: #dalla colonna 2
    print(f"Entropy for column {c} : {entropy(df[c])}")

Entropy for column age_gt_median : 0.9967916319816366
Entropy for column parmigiana : 0.8366407419411673
Entropy for column polenta : 0.9709505944546686
Entropy for column sushi : 0.9709505944546686
Entropy for column meridionale : 0.9182958340544896


In [23]:
def get_information_gain(data, split_name, target_name): #per un certo dataset, su un certo split e su un certo dataset
    original_entropy=entropy(data[target_name])
    print(original_entropy)

    values=data[split_name].unique() #sono i valori contenuti nella nostra colonna
    print(values)

    #tutti i valori per quella colonna che sono uguali a 1
    left_split= data[data[split_name]==values[0]]
    right_split=data[data[split_name]==values[1]]

    to_subtract=0   

    #prendiamo il numero di entry del subset e lo divisiamo per il numero totale di elementi nel dataset
    for subset in [left_split, right_split]:
        prob=(subset.shape[0]/data.shape[0])
        to_subtract +=prob*entropy(subset[target_name])

    return original_entropy-to_subtract

get_information_gain(df, "age_gt_median", "meridionale")

0.9182958340544896
[1 0]


0.006474767163413775

In [25]:
def max_info_gain(df, columns, target): 
    #d={
    #   "col_name":ig_val-1,
    #   "col_name_2":ig_val_2
    #}

    gains={ #dizionario che contiene tutti i valiri dell'information gain

    }
    for c in columns:
        inf_gain=get_information_gain(df, c, target)
        gains[c]= inf_gain #entry nel dizionario
    
    return max(gains, key=gains.get) #voglio che mi ritorni la colonna con l'information gain maggiore

In [28]:
max_info_gain(df, df.columns[2:-1], "meridionale") #target meridionale, quindi la tolgo

0.9182958340544896
[1 0]
0.9182958340544896
[0 1]
0.9182958340544896
[1 0]
0.9182958340544896
[1 0]


'sushi'